views.py

In [ ]:
from django.shortcuts import render, redirect
from django.http import Http404
from django.urls import reverse
from .models import Week, Choice
# import the login_required decorator and the LoginRequiredMixin mixin below
from django.contrib.auth.decorators import login_required
from django.contrib.auth.mixins import LoginRequiredMixin
# Import the logout function from django.contrib.auth below
from django.contrib.auth import logout
from django.urls import reverse_lazy
from django.contrib.auth.forms import UserCreationForm
from django.views.generic import ListView, DetailView
from django.views.generic.edit import CreateView

# Add login_required decorator:
@login_required
def index(request):
  latest_week_poll = Week.objects.get(pk=1)
  context = {
     "name": request.user,
    "week_poll": latest_week_poll
  }
  return render(request, "index.html", context)

# Create your class-based Signup view below:
class SignUp(CreateView):
  form_class = UserCreationForm
  success_url = reverse_lazy("login")
  template_name = "registration/signup.html"

class DetailsView(LoginRequiredMixin, DetailView):
  model = Week
  template_name = "detail.html"

class ResultsView(LoginRequiredMixin, DetailView):
  model = Week
  template_name = "results.html"

# Create your logout function, logout_request, below:
def logout_request(request):
  logout(request)
  return redirect("index")

# Add login_required decorator:
@login_required
def vote(request, week_id):
  try:
    week = Week.objects.get(pk=week_id)
    selected_choice = week.choice_set.get(pk=request.POST["choice"])
  except Week.DoesNotExist:
    raise Http404("Poll for said week does not exists")
  else:
    selected_choice.votes += 1
    selected_choice.save()
    return redirect("results", week_id)


urls.py

In [ ]:
from django.urls import path, include

from . import views

urlpatterns = [
	path("", views.index, name="index"),
  # Add a path to your auth links below:
  path("account/", include("django.contrib.auth.urls")),
	# Add a path to your signup request below:
  path("signup/", views.SignUp.as_view(), name="signup"),
	# Add a path to your logout request below:
  path("logout/", views.logout, name="logout")
	path("polls/<int:pk>/", views.DetailsView.as_view(), name="detail"),
	path("polls/<int:pk>/results/", views.ResultsView.as_view(), name="results"),
	path("polls/<int:week_id>/vote/", views.vote, name="vote"),
]

models.py

In [ ]:
import datetime
from django.utils import timezone
from django.db import models

class Week(models.Model):
  pub_date = models.DateTimeField("date published")

  # When creating a new instance, we want to inform students of the menu for the following week starting on Monday:
  def start_of_week(self):
		# Using the module `datetime`, we can take todays date:
    today = datetime.date.today()
		# Subtract the number of days which already passed this week (this gets you 'last' monday).
		# And add one week
    start_of_week = today + datetime.timedelta(days=-today.weekday(), weeks=1)
    return start_of_week

  def was_published_recently(self):
    return self.pub_date >= timezone.now() - datetime.timedelta(days=1)


class Choice(models.Model):
  week = models.ForeignKey(Week, on_delete=models.CASCADE)
  dessert_choice = models.CharField(max_length=200)
  dessert_details = models.CharField(max_length=200)
  votes = models.IntegerField(default=0)
  def __str__(self):
    return self.dessert_choice

header.html

In [ ]:
{% load static %}

<link rel="stylesheet" type="text/css" href="{% static 'weeklydessert/header.css' %}">

<div class="header">
  <img src="{% static 'weeklydessert/Dessert.png' %}" >
	{% if user.username %}
	  <a class="logout-button" href="{% url 'logout' %}">Log Out</a>
  {% endif %}
  <div class="header-content">
    <h1 class="header-title">Dessert Island</h1>
    {% if user.username %}
    <h4 class="header-greet">Welcome {{ user.username }}!</h4>
    {% endif %}
  </div>
</div>

{% block content%}
{% endblock %}

login.htnl

In [ ]:
{% extends "header.html" %}

{% block content %}
{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'weeklydessert/login.css' %}">

<div class="login-form">
<h2>Please login</h2>
  <form class="login" method="post">
      <div>
        {% csrf_token %}
          {{ form.as_p }}
          
        <input type="submit" value="Login"/>
      </div>
  </form>
</div>
{% endblock %}